<a href="https://colab.research.google.com/github/martin-fabbri/colab-notebooks/blob/master/deeplearning.ai/gan/c1_w3_wgan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wasserstein GAN with Gradient Penalty (WGAN-GP)

## Motivation

Training GANs is hard. Models may never converge and mode collapses are common. To move forward, we need to make incremental improvements or embrace optimal cost functions.

### WGAN-GP improvements

WGAN-GP enhances training stability. As shown below, when the model design is less optimal, WGAN-GP can still create good results while the original GAN cost function fails.

<img src="https://github.com/martin-fabbri/colab-notebooks/raw/master/deeplearning.ai/gan/images/wgan-gp-1.png" width=600>


Below is the inception score using different methods. The experiment from WGAN-GP paper demostrates better image quality and convergence comparing with WGAN. However, DCGAN demostrates slightly better image quality and it converges faster. But the inception score for WGAN-GP is more stable when it starts converting.

<img src="https://github.com/martin-fabbri/colab-notebooks/raw/master/deeplearning.ai/gan/images/wgan-gp-2.png" width=500>

> ### So what is the benefit of WGAN-GP if it cannot beat DCGAN?
> - `The major advantage of WGAN-GP is its convergency`. It makes training more stable and therefore easier to train. As WGAN-GP helps model to converge better, we can use a more complex model lie deep ResNet for generation and the discriminator.  
> - When using the Wasserstein loss function, we should train the critic to `converge to ensure that the gradients for the generator are accurate`. This is `in contrast to a standard GAN`, where it is important `not to let the discrimininator get too strong, to avoid vanishing gradients`. Helping to reduce one of the main difficulties of training GANs - How to balance the training of the discriminator and generator.

## Wasserstein Distance

Wasserstein GAN (WGAN) proposes a new cost function using Wasserstein distance that has a smoother gradient everyware. WGAN learns regarless of the generator performance. The diagram below repeats a similar plot on the value $D(X)$ fro both GAN and WGAN. For GAN (the red line), it fills with areas with diminishing or exploiding gradient. For WANG (the blue line), the gradient is smoother everywhere and learns better even when the generator is not producing good images.

<img src="https://github.com/martin-fabbri/colab-notebooks/raw/master/deeplearning.ai/gan/images/wgan-4.png" width=300>



GAN `discriminator loss` minimization $$min_D−(\mathbb{E}_{x∼pX}[log\,D(x)] + \mathbb{E}_{z∼pZ}[log\,(1−D(G(z)))])$$

To train the GAN `generator` G, we calculate the loss when comparing predictions for generated images $p_i=D(G(zi))$ to the response y_i=1. Therefore for the GAN generator, minimizing the loss function can be written as follows:

GAN `generator loss` minimization:
$$min_G−(\mathbb{E}_{z∼pZ}[log\,D(G(z))])$$

### Wasserstein loss function

First, the Wasserstein loss requires that we use $y_i=1$ and $y_i=-1$ as labels, rather than 1 and 0. We also remove the sigmoid activation from the final layer of the **discriminator(critic)**, so that predictions $p_i$ are no longer constrained to fall in the range $[0,1]$, but instead can now be any number in the range $[–∞, ∞]$. For this reason, the discriminator in a WGAN is usually referred to as a critic.The Wasserstein loss function is then defined as follows:

Wasserstein loss:
$$-\frac{1}{n}\sum_{i=1}^{n}{(y_i\,p_i)}$$

To train the WGAN critic $D$, we calculate the loss when comparing predictions for a real images $p_i=D(x_i)$ to the response $y_i=1$ and predictions for generated images $p_i=D(G(z_i))$ to the response $y_i=-1$. Therefore for the WGAN critic, minimizing the loss function can be written as follows:

WGAN critic loss minimization:

$$min_D−(\mathbb{E}_{x∼pX}[D(x)]−\mathbb{E}_{z∼pZ}[D(G(z))])$$

### The Lipschitz Constraint

It may surprise you that we are now allowing the critic to output any number in the range $[–∞, ∞]$, rather than applying a sigmoid function to restrict the output to the usual $[0, 1]$ range. The Wasserstein loss can therefore be very large, which is unsettling—usually, large numbers in neural networks are to be avoided!

In fact, the authors of the WGAN paper show that for the Wasserstein loss function to work, we also need to place an additional constraint on the critic. Specifically, it is required that the critic is a `1-Lipschitz continuous` function. Let’s pick this apart to understand what it means in more detail.

The critic is a function $D$ that converts an image into a prediction. We say that this function is 1-Lipschitz if it satisfies the following inequality for any two input images x1 and x2:

$$\frac{|D(x_1)−D(x_2)|}{|x_1−x_2|}\leq1$$

Here, $x_1 – x_2$ is the average pixelwise absolute difference between two images and $|D(x_1)−D(x_2)||D(x_1)-D(x_2)|$ is the absolute difference between the critic predictions. Essentially, we require a limit on the rate at which the predictions of the critic can change between two images (i.e., the absolute value of the gradient must be at most 1 everywhere).

## Key differences between GANs and WGANs

- A WGAN uses the Wasserstein loss.

- The WGAN is trained using labels of `1 for real` and `–1 for fake`.

- There is `no need for the sigmoid activation` in the final layer of the WGAN critic.

- `Clip the weights` of the critic after each update.

- Train the critic multiple times for each update of the generator.

### Goals
In this notebook, you're going to build a Wasserstein GAN with Gradient Penalty (WGAN-GP) that solves some of the stability issues with the GANs that you have been using up until this point. Specifically, you'll use a special kind of loss function known as the W-loss, where W stands for Wasserstein, and gradient penalties to prevent mode collapse.

*Fun Fact: Wasserstein is named after a mathematician at Penn State, Leonid Vaseršteĭn. You'll see it abbreviated to W (e.g. WGAN, W-loss, W-distance).*

### Learning Objectives
1.   Get hands-on experience building a more stable GAN: Wasserstein GAN with Gradient Penalty (WGAN-GP).
2.   Train the more advanced WGAN-GP model.

## Generator and Critic

You will begin by importing some useful packages, defining visualization functions, building the generator, and building the critic. Since the changes for WGAN-GP are done to the loss function during training, you can simply reuse your previous GAN code for the generator and critic class. Remember that in WGAN-GP, you no longer use a discriminator that classifies fake and real as 0 and 1 but rather a critic that scores images with real numbers.

In [3]:
import torch
import matplotlib.pyplot as plt

from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import make_grid
from torch.utils.data import DataLoader

torch.manual_seed(0)
torch.__version__

'1.7.0+cu101'

In [4]:
def show_tensor_images(image_tensor, num_images=25, size=(1, 28, 28)):
    '''
    Function for visualizing images: Given a tensor of images, number of images, and
    size per image, plots and prints the images in an uniform grid.
    '''
    image_tensor = (image_tensor + 1) / 2
    image_unflat = image_tensor.detach().cpu()
    image_grid = make_grid(image_unflat[:num_images], nrow=5)
    plt.imshow(image_grid.permute(1, 2, 0).squeeze())
    plt.show()

def make_grad_hook():
    '''
    Function to keep track of gradients for visualization purposes, 
    which fills the grads list when using model.apply(grad_hook).
    '''
    grads = []
    def grad_hook(m):
        if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
            grads.append(m.weight.grad)
    return grads, grad_hook

### Generator and Noise

In [5]:
def get_noise(n_samples, z_dim, device='cpu'):
  '''
  Function for creating noise vectors: Given the dimensions (n_samples, z_dim)
  creates a tensor of that shape filled with random numbers from the normal distribution.
  Parameters:
    n_samples: the number of samples to generate, a scalar
    z_dim: the dimension of the noise vector, a scalar
    device: the device type
  '''
  return torch.randn(n_samples, z_dim, device=device)

In [22]:
class Generator(nn.Module):
  '''
  Generator Class
  Values:
      z_dim: the dimension of the noise vector, a scalar
      im_chan: the number of channels in the images, fitted for the dataset used, a scalar
            (MNIST is black-and-white, so 1 channel is your default)
      hidden_dim: the inner dimension, a scalar
  '''

  def __init__(self, z_dim=10, im_chan=1, hidden_dim=64):
    super(Generator, self).__init__()
    self.z_dim = z_dim
    # build the neural network
    self.gen = nn.Sequential(
      self.make_gen_block(z_dim, hidden_dim * 4),
      self.make_gen_block(hidden_dim * 4, hidden_dim * 2, kernel_size=4, 
                          stride=1),
      self.make_gen_block(hidden_dim * 2, hidden_dim),                    
      self.make_gen_block(hidden_dim, im_chan, kernel_size=4, 
                          final_layer=True)
    )
  
  def make_gen_block(self, input_channels, output_channels, kernel_size=3, 
                     stride=2, final_layer=False):
    '''
    Function to return a sequence of operations corresponding to a generator block of DCGAN;
    a transposed convolution, a batchnorm (except in the final layer), and an activation.
    Parameters:
        input_channels: how many channels the input feature representation has
        output_channels: how many channels the output feature representation should have
        kernel_size: the size of each convolutional filter, equivalent to (kernel_size, kernel_size)
        stride: the stride of the convolution
        final_layer: a boolean, true if it is the final layer and false otherwise 
                  (affects activation and batchnorm)
    '''
    if not final_layer:
      return nn.Sequential(
          nn.ConvTranspose2d(input_channels, output_channels, kernel_size, 
                             stride),
          nn.BatchNorm2d(output_channels),
          nn.ReLU(inplace=True),
      )
    else:
      return nn.Sequential(
          nn.ConvTranspose2d(input_channels, output_channels, kernel_size, 
                             stride),
          nn.Tanh(),
      )

  def forward(self, noise):
    '''
    Function for completing a forward pass of the generator: Given a noise tensor,
    returns generated images.
    Parameters:
        noise: a noise tensor with dimensions (n_samples, z_dim)
    '''
    x = noise.view(len(noise), self.z_dim, 1, 1)
    return self.gen(x)

### Critic


In [33]:
class Critic(nn.Module):
  '''
  Critic Class
  Values:
      im_chan: the number of channels in the images, fitted for the dataset used, a scalar
            (MNIST is black-and-white, so 1 channel is your default)
      hidden_dim: the inner dimension, a scalar
  '''
  def __init__(self, im_chan=1, hidden_dim=64):
    super(Critic, self).__init__()
    self.crit = nn.Sequential(
        self.make_crit_block(im_chan, hidden_dim),
        self.make_crit_block(hidden_dim, hidden_dim * 2),
        self.make_crit_block(hidden_dim * 2, 1, final_layer=True)
    )

  def make_crit_block(self, input_channels, output_channels, kernel_size=4, 
                      stride=2, final_layer=False):
    '''
    Function to return a sequence of operations corresponding to a critic block of DCGAN;
    a convolution, a batchnorm (except in the final layer), and an activation (except in the final layer).
    Parameters:
        input_channels: how many channels the input feature representation has
        output_channels: how many channels the output feature representation should have
        kernel_size: the size of each convolutional filter, equivalent to (kernel_size, kernel_size)
        stride: the stride of the convolution
        final_layer: a boolean, true if it is the final layer and false otherwise 
                  (affects activation and batchnorm)
    '''
    if not final_layer:
      return nn.Sequential(
          nn.Conv2d(input_channels, output_channels, kernel_size, stride),
          nn.BatchNorm2d(output_channels),
          nn.LeakyReLU(0.2, inplace=True)
      )
    else:
      return nn.Sequential(
          nn.Conv2d(input_channels, output_channels, kernel_size, stride)
      )

  def forward(self, image):
    '''
    Function for completing a forward pass of the critic: Given an image tensor, 
    returns a 1-dimension tensor representing fake/real.
    Parameters:
        image: a flattened image tensor with dimension (im_chan)
    '''
    crit_pred = self.crit(image)
    return crit_pred.view(len(crit_pred), -1)

## Training Initializations
Now you can start putting it all together.
As usual, you will start by setting the parameters:
  *   n_epochs: the number of times you iterate through the entire dataset when training
  *   z_dim: the dimension of the noise vector
  *   display_step: how often to display/visualize the images
  *   batch_size: the number of images per forward/backward pass
  *   lr: the learning rate
  *   beta_1, beta_2: the momentum terms
  *   c_lambda: weight of the gradient penalty
  *   crit_repeats: number of times to update the critic per generator update - there are more details about this in the *Putting It All Together* section
  *   device: the device type

You will also load and transform the MNIST dataset to tensors.

In [34]:
n_epochs = 100
z_dim = 64
display_step = 50
batch_size = 128
lr = 0.0002
beta_1 = 0.5
beta_2 = 0.999
c_lambda = 10
crit_repeats = 5
device = 'cuda'

In [35]:
transform = transforms.Compose([
  transforms.ToTensor(),
  transforms.Normalize((0.5,), (0.5,))
])

dataloader = DataLoader(
    MNIST('.', download=True, transform=transform),
    batch_size=batch_size,
    shuffle=True
)

Then, you can initialize your generator, critic, and optimizers.

In [39]:
def weights_init(m):
  if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
      torch.nn.init.normal_(m.weight, 0.0, 0.02)
  if isinstance(m, nn.BatchNorm2d):
      torch.nn.init.normal_(m.weight, 0.0, 0.02)
      torch.nn.init.constant_(m.bias, 0)

In [40]:
gen = Generator(z_dim).to(device)
gen = gen.apply(weights_init)
crit = Critic().to(device)
crit = crit.apply(weights_init)

get_opt = torch.optim.Adam(gen.parameters(), lr=lr, betas=(beta_1, beta_2))
crit_opt = torch.optim.Adam(crit.parameters(), lr=lr, betas=(beta_1, beta_2))

## Gradient Penalty
Calculating the gradient penalty can be broken into two functions: (1) compute the gradient with respect to the images and (2) compute the gradient penalty given the gradient.

You can start by getting the gradient. The gradient is computed by first creating a mixed image. This is done by weighing the fake and real image using epsilon and then adding them together. Once you have the intermediate image, you can get the critic's output on the image. Finally, you compute the gradient of the critic score's on the mixed images (output) with respect to the pixels of the mixed images (input). You will need to fill in the code to get the gradient wherever you see *None*. There is a test function in the next block for you to test your solution.

In [43]:
# UNQ_C1 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
# GRADED FUNCTION: get_gradient
def get_gradient(crit, real, fake, epsilon):
    '''
    Return the gradient of the critic's scores with respect to mixes of real and fake images.
    Parameters:
        crit: the critic model
        real: a batch of real images
        fake: a batch of fake images
        epsilon: a vector of the uniformly random proportions of real/fake per mixed image
    Returns:
        gradient: the gradient of the critic's scores, with respect to the mixed image
    '''
    # Mix the images together
    mixed_images = real * epsilon + fake * (1 - epsilon)

    # Calculate the critic's scores on the mixed images
    mixed_scores = crit(mixed_images)
    
    # Take the gradient of the scores with respect to the images
    gradient = torch.autograd.grad(
        # Note: You need to take the gradient of outputs with respect to inputs.
        # This documentation may be useful, but it should not be necessary:
        # https://pytorch.org/docs/stable/autograd.html#torch.autograd.grad
        #### START CODE HERE ####
        inputs=mixed_images,
        outputs=mixed_scores,
        #### END CODE HERE ####
        # These other parameters have to do with the pytorch autograd engine works
        grad_outputs=torch.ones_like(mixed_scores), 
        create_graph=True,
        retain_graph=True,
    )[0]
    return gradient

In [44]:
# UNIT TEST
# DO NOT MODIFY THIS
def test_get_gradient(image_shape):
    real = torch.randn(*image_shape, device=device) + 1
    fake = torch.randn(*image_shape, device=device) - 1
    epsilon_shape = [1 for _ in image_shape]
    epsilon_shape[0] = image_shape[0]
    epsilon = torch.rand(epsilon_shape, device=device).requires_grad_()
    gradient = get_gradient(crit, real, fake, epsilon)
    assert tuple(gradient.shape) == image_shape
    assert gradient.max() > 0
    assert gradient.min() < 0
    return gradient

gradient = test_get_gradient((256, 1, 28, 28))
print("Success!")

Success!


The second function you need to complete is to compute the gradient penalty given the gradient. First, you calculate the magnitude of each image's gradient. The magnitude of a gradient is also called the norm. Then, you calculate the penalty by squaring the distance between each magnitude and the ideal norm of 1 and taking the mean of all the squared distances.

Again, you will need to fill in the code wherever you see *None*. There are hints below that you can view if you need help and there is a test function in the next block for you to test your solution.

<details>

<summary>
<font size="3" color="green">
<b>Optional hints for <code><font size="4">gradient_penalty</font></code></b>
</font>
</summary>


1.   Make sure you take the mean at the end.
2.   Note that the magnitude of each gradient has already been calculated for you.

</details>

In [45]:
# UNQ_C2 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
# GRADED FUNCTION: gradient_penalty
def gradient_penalty(gradient):
    '''
    Return the gradient penalty, given a gradient.
    Given a batch of image gradients, you calculate the magnitude of each image's gradient
    and penalize the mean quadratic distance of each magnitude to 1.
    Parameters:
        gradient: the gradient of the critic's scores, with respect to the mixed image
    Returns:
        penalty: the gradient penalty
    '''
    # Flatten the gradients so that each row captures one image
    gradient = gradient.view(len(gradient), -1)

    # Calculate the magnitude of every row
    gradient_norm = gradient.norm(2, dim=1)
    
    # Penalize the mean squared distance of the gradient norms from 1
    #### START CODE HERE ####
    penalty = torch.mean(torch.square(1 - gradient_norm))
    #### END CODE HERE ####
    return penalty

In [46]:
# UNIT TEST
def test_gradient_penalty(image_shape):
    bad_gradient = torch.zeros(*image_shape)
    bad_gradient_penalty = gradient_penalty(bad_gradient)
    assert torch.isclose(bad_gradient_penalty, torch.tensor(1.))

    image_size = torch.prod(torch.Tensor(image_shape[1:]))
    good_gradient = torch.ones(*image_shape) / torch.sqrt(image_size)
    good_gradient_penalty = gradient_penalty(good_gradient)
    assert torch.isclose(good_gradient_penalty, torch.tensor(0.))

    random_gradient = test_get_gradient(image_shape)
    random_gradient_penalty = gradient_penalty(random_gradient)
    assert torch.abs(random_gradient_penalty - 1) < 0.1

test_gradient_penalty((256, 1, 28, 28))
print("Success!")

Success!


## Losses
Next, you need to calculate the loss for the generator and the critic.

For the generator, the loss is calculated by maximizing the critic's prediction on the generator's fake images. The argument has the scores for all fake images in the batch, but you will use the mean of them.

There are optional hints below and a test function in the next block for you to test your solution.

<details><summary><font size="3" color="green"><b>Optional hints for <code><font size="4">get_gen_loss</font></code></b></font></summary>

1. This can be written in one line.
2. This is the negative of the mean of the critic's scores.

</details>

In [47]:
# UNQ_C3 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
# GRADED FUNCTION: get_gen_loss
def get_gen_loss(crit_fake_pred):
    '''
    Return the loss of a generator given the critic's scores of the generator's fake images.
    Parameters:
        crit_fake_pred: the critic's scores of the fake images
    Returns:
        gen_loss: a scalar loss value for the current batch of the generator
    '''
    #### START CODE HERE ####
    gen_loss = -torch.mean(crit_fake_pred)
    #### END CODE HERE ####
    return gen_loss

In [48]:
# UNIT TEST
assert torch.isclose(
    get_gen_loss(torch.tensor(1.)), torch.tensor(-1.0)
)

assert torch.isclose(
    get_gen_loss(torch.rand(10000)), torch.tensor(-0.5), 0.05
)

print("Success!")

Success!


For the critic, the loss is calculated by maximizing the distance between the critic's predictions on the real images and the predictions on the fake images while also adding a gradient penalty. The gradient penalty is weighed according to lambda. The arguments are the scores for all the images in the batch, and you will use the mean of them.

There are hints below if you get stuck and a test function in the next block for you to test your solution.

<details><summary><font size="3" color="green"><b>Optional hints for <code><font size="4">get_crit_loss</font></code></b></font></summary>

1. The higher the mean fake score, the higher the critic's loss is.
2. What does this suggest about the mean real score?
3. The higher the gradient penalty, the higher the critic's loss is, proportional to lambda.


</details>

In [56]:
# UNQ_C4 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
# GRADED FUNCTION: get_crit_loss
def get_crit_loss(crit_fake_pred, crit_real_pred, gp, c_lambda):
    '''
    Return the loss of a critic given the critic's scores for fake and real images,
    the gradient penalty, and gradient penalty weight.
    Parameters:
        crit_fake_pred: the critic's scores of the fake images
        crit_real_pred: the critic's scores of the real images
        gp: the unweighted gradient penalty
        c_lambda: the current weight of the gradient penalty 
    Returns:
        crit_loss: a scalar for the critic's loss, accounting for the relevant factors
    '''
    #### START CODE HERE ####
    crit_loss = torch.mean(crit_fake_pred) - torch.mean(crit_real_pred) + c_lambda * torch.mean(gp)
    #### END CODE HERE ####
    return crit_loss

In [57]:
# UNIT TEST
assert torch.isclose(
    get_crit_loss(torch.tensor(1.), torch.tensor(2.), torch.tensor(3.), 0.1),
    torch.tensor(-0.7)
)
assert torch.isclose(
    get_crit_loss(torch.tensor(20.), torch.tensor(-20.), torch.tensor(2.), 10),
    torch.tensor(60.)
)

print("Success!")

Success!
